# 🎯 Prompt Templates: A Arte de Fazer Perguntas Inteligentes para IA

**Pedro Nunes Guth | Módulo 4 de 17 - LangChain v0.3**

![](/imagens/langchain-modulo-04_img_01.png)

## 🚀 Bora entender o que são Prompt Templates!

Tá, mas o que é um **Prompt Template**? 

Imagina que você trabalha numa padaria e todo dia chega um cliente diferente pedindo um pão na choca. Ao invés de você inventar uma receita nova toda vez, você tem um template:

- "Oi {nome_cliente}, seu pão {tipo_pao} com {recheio} ficará pronto em {tempo} minutos!"

O **Prompt Template** é exatamente isso! É um molde reutilizável onde você define a estrutura da sua conversa com a IA e só muda as variáveis.

### Por que isso é LIIINDO?

1. **Consistência**: Mesmo resultado toda vez
2. **Reutilização**: Escreve uma vez, usa mil vezes
3. **Manutenção**: Mudou algo? Altera só no template
4. **Organização**: Seu código fica limpo e profissional

**Dica!** No módulo anterior vimos Runnables e LCEL. Os templates se conectam perfeitamente com eles usando o operador `|` (pipe)!

In [ ]:
# Bora instalar e importar o que precisamos!
!pip install langchain langchain-google-genai python-dotenv -q

import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Carregando variáveis de ambiente
load_dotenv()

print("✅ Tudo importado! Bora começar!")

In [ ]:
# Configurando nossa IA (lembrando do módulo 2!)
os.environ["GOOGLE_API_KEY"] = "sua_api_key_aqui"  # Substitua pela sua chave

# Inicializando o modelo que já conhecemos
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.1
)

print("🤖 Gemini conectado e pronto para usar templates!")

## 📝 PromptTemplate: O Básico dos Básicos

O `PromptTemplate` é como aquela receita de bolo da vovó: simples, funciona sempre e todo mundo entende!

### Anatomia de um Template:

```python
template = "Você é um {papel} especialista em {area}. {instrucao}"
```

As palavras entre `{}` são **variáveis** que você vai preencher depois. É tipo um formulário!

**Dica!** Sempre pense no template como uma conversa que você teria com um especialista. Seja claro no que você quer!

In [ ]:
# Nosso primeiro template simples
template_simples = "Explique {conceito} de forma simples para uma pessoa de {idade} anos."

# Criando o PromptTemplate
prompt = PromptTemplate(
    input_variables=["conceito", "idade"],
    template=template_simples
)

# Vamos ver como fica formatado
prompt_formatado = prompt.format(conceito="inteligência artificial", idade=10)
print("📝 Prompt formatado:")
print(prompt_formatado)

In [ ]:
# Agora bora usar com nossa IA (usando LCEL do módulo 3!)
chain = prompt | llm

# Testando com diferentes idades
idades_teste = [5, 15, 25]

for idade in idades_teste:
    resposta = chain.invoke({
        "conceito": "blockchain", 
        "idade": idade
    })
    
    print(f"\n🎯 Para {idade} anos:")
    print(resposta.content[:200] + "...")
    print("-" * 50)

## 💬 ChatPromptTemplate: Para Conversas Estruturadas

Tá, mas e se eu quiser ter uma conversa mais estruturada? Tipo definir quem é o **system**, o **user** e até exemplos de **assistant**?

É aí que entra o `ChatPromptTemplate`! É como montar um roteiro de teatro onde cada ator tem seu papel:

- **System**: As regras do jogo, o contexto
- **Human/User**: O que o usuário pergunta  
- **Assistant**: Exemplos de como a IA deve responder

![](/imagens/langchain-modulo-04_img_02.png)

**Dica!** Use o system message para definir a personalidade da IA. Quer que ela seja formal? Divertida? Técnica? É no system que você define!

In [ ]:
# Criando um ChatPromptTemplate mais elaborado
chat_template = ChatPromptTemplate.from_messages([
    ("system", "Você é {persona}, um especialista em {area}. Responda sempre de forma {estilo}."),
    ("human", "Olá! Eu tenho {nivel_experiencia} em {area}."),
    ("assistant", "Ótimo! Vou ajudar você considerando seu nível. O que gostaria de saber?"),
    ("human", "{pergunta}")
])

print("📋 Template criado com múltiplas mensagens!")
print(f"Variables: {chat_template.input_variables}")

In [ ]:
# Vamos testar com diferentes personas
personas_teste = [
    {
        "persona": "Dr. Python", 
        "area": "programação Python",
        "estilo": "didática e com exemplos práticos",
        "nivel_experiencia": "iniciante",
        "pergunta": "Como funcionam as listas em Python?"
    },
    {
        "persona": "Chef Código", 
        "area": "culinária",
        "estilo": "divertida usando analogias com programação",
        "nivel_experiencia": "intermediário",
        "pergunta": "Como fazer um risotto perfeito?"
    }
]

chat_chain = chat_template | llm

for i, params in enumerate(personas_teste, 1):
    print(f"\n🎭 Teste {i}: {params['persona']}")
    resposta = chat_chain.invoke(params)
    print(resposta.content[:300] + "...")
    print("=" * 60)

## 🎨 Templates Avançados: Formatação e Validação

Agora vamos incrementar! Imagina que você quer garantir que certas variáveis sempre tenham valores válidos, ou que o template seja mais dinâmico.

### Recursos Avançados:

1. **Validação de inputs**: Garante que você não esqueça nenhuma variável
2. **Templates parciais**: Predefine algumas variáveis
3. **Formatação condicional**: Muda o template baseado nos inputs

**Dica!** Pense nos templates como contratos. Quanto mais claro o que entra e sai, menos bugs você vai ter!

In [ ]:
# Template para análise de código com validação
template_codigo = """
Você é um revisor de código especialista em {linguagem}.

CONTEXTO:
- Nível do programador: {nivel}
- Tipo de análise: {tipo_analise}
- Foco especial: {foco}

CÓDIGO PARA ANALISAR:
```{linguagem}
{codigo}
```

INSTRUÇÕES:
1. Analise o código considerando o nível {nivel}
2. Foque especialmente em: {foco}
3. Dê sugestões práticas e aplicáveis
4. Use exemplos se necessário
"""

prompt_codigo = PromptTemplate(
    input_variables=["linguagem", "nivel", "tipo_analise", "foco", "codigo"],
    template=template_codigo
)

print("✅ Template de análise de código criado!")
print(f"Variáveis necessárias: {prompt_codigo.input_variables}")

In [ ]:
# Testando com um código Python simples
codigo_exemplo = """
def calcular_media(numeros):
    soma = 0
    for numero in numeros:
        soma = soma + numero
    media = soma / len(numeros)
    return media

lista_notas = [8, 7, 9, 6, 8]
resultado = calcular_media(lista_notas)
print(resultado)
"""

parametros_analise = {
    "linguagem": "Python",
    "nivel": "iniciante",
    "tipo_analise": "melhores práticas",
    "foco": "legibilidade e eficiência",
    "codigo": codigo_exemplo
}

# Criando a chain e executando
codigo_chain = prompt_codigo | llm
analise = codigo_chain.invoke(parametros_analise)

print("🔍 ANÁLISE DO CÓDIGO:")
print(analise.content)

## 📊 Visualizando o Fluxo dos Templates

Bora visualizar como os templates funcionam na prática? É importante entender o fluxo completo!

```mermaid
graph TD
    A[Template String] --> B[Input Variables]
    B --> C[PromptTemplate.format]
    C --> D[Formatted Prompt]
    D --> E[LLM]
    E --> F[Response]
    
    style A fill:#e1f5fe
    style C fill:#f3e5f5
    style E fill:#fff3e0
    style F fill:#e8f5e8
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Simulando dados de performance de diferentes tipos de templates
tipos_template = ['Simples', 'Chat', 'Estruturado', 'Validado']
facilidade_uso = [9, 7, 6, 5]
flexibilidade = [4, 6, 8, 9]
confiabilidade = [5, 7, 8, 9]

x = np.arange(len(tipos_template))
width = 0.25

fig, ax = plt.subplots(figsize=(12, 6))

bars1 = ax.bar(x - width, facilidade_uso, width, label='Facilidade de Uso', color='#FF6B6B')
bars2 = ax.bar(x, flexibilidade, width, label='Flexibilidade', color='#4ECDC4')
bars3 = ax.bar(x + width, confiabilidade, width, label='Confiabilidade', color='#45B7D1')

ax.set_xlabel('Tipos de Template')
ax.set_ylabel('Score (1-10)')
ax.set_title('📊 Comparação de Tipos de Prompt Templates')
ax.set_xticks(x)
ax.set_xticklabels(tipos_template)
ax.legend()
ax.grid(True, alpha=0.3)

# Adicionando valores nas barras
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{height}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("💡 Dica: Comece simples e vá evoluindo conforme a necessidade!")

## 🔧 Templates Parciais: Predefina o que Não Muda

Tá, imagina que você tem um template onde algumas variáveis nunca mudam. Por exemplo, você sempre usa a mesma linguagem ou sempre quer o mesmo estilo.

Ao invés de ficar passando a mesma coisa toda vez, você pode usar **templates parciais**!

É como ter um carimbo já meio preenchido - você só completa o que falta.

**Dica!** Use templates parciais quando você tem configurações padrão que raramente mudam!

In [ ]:
# Template para tutoriais técnicos
template_tutorial = """
Você é um instrutor de {tecnologia} com {anos_experiencia} anos de experiência.
Seu estilo é sempre {estilo} e você foca em {publico_alvo}.

TÓPICO: {topico}
DIFICULDADE: {dificuldade}

Crie um mini-tutorial sobre: {topico}
Inclua:
1. Explicação conceitual
2. Exemplo prático
3. Dica importante
"""

# Criando o template base
prompt_tutorial = PromptTemplate(
    input_variables=["tecnologia", "anos_experiencia", "estilo", "publico_alvo", "topico", "dificuldade"],
    template=template_tutorial
)

# Agora vamos criar um template parcial com nossas configurações padrão
prompt_python_guth = prompt_tutorial.partial(
    tecnologia="Python",
    anos_experiencia="10",
    estilo="descontraído mas técnico",
    publico_alvo="iniciantes em programação"
)

print("✅ Template parcial criado!")
print(f"Variáveis restantes: {prompt_python_guth.input_variables}")

In [ ]:
# Testando o template parcial - agora só precisamos de 2 variáveis!
topicos_python = [
    {"topico": "list comprehensions", "dificuldade": "intermediário"},
    {"topico": "decorators", "dificuldade": "avançado"},
    {"topico": "loops for", "dificuldade": "básico"}
]

tutorial_chain = prompt_python_guth | llm

for i, params in enumerate(topicos_python, 1):
    print(f"\n📚 Tutorial {i}: {params['topico']}")
    tutorial = tutorial_chain.invoke(params)
    print(tutorial.content[:400] + "...")
    print("=" * 70)

## 🎯 Templates para Few-Shot Learning

Agora uma técnica LINDA! **Few-Shot Learning** é quando você dá alguns exemplos para a IA entender o padrão que você quer.

É como ensinar uma criança mostrando: "Olha, quando eu falo isso, você responde assim. Quando falo aquilo, responde assado."

![](/imagens/langchain-modulo-04_img_03.png)

**Dica!** Quanto melhores seus exemplos, melhor a IA vai entender o que você quer!

In [ ]:
# Template para classificação de sentimentos com exemplos
template_sentimento = """
Classifique o sentimento dos textos como: POSITIVO, NEGATIVO ou NEUTRO.

Exemplos:

Texto: "Adorei o atendimento, voltarei sempre!"
Sentimento: POSITIVO

Texto: "O produto chegou quebrado e o suporte não resolve."
Sentimento: NEGATIVO

Texto: "O pedido chegou no prazo informado."
Sentimento: NEUTRO

Texto: "Comprei ontem e recebi hoje, mas ainda não testei."
Sentimento: NEUTRO

Agora classifique:

Texto: "{texto}"
Sentimento:
"""

prompt_sentimento = PromptTemplate(
    input_variables=["texto"],
    template=template_sentimento
)

print("🎯 Template Few-Shot para análise de sentimento criado!")

In [ ]:
# Testando com vários textos
textos_teste = [
    "Este curso de Python está sendo incrível! Aprendendo muito!",
    "Não consegui instalar o LangChain, muito complicado.",
    "O módulo 4 foi sobre prompt templates.",
    "Finalmente entendi como usar IA na prática! Obrigado Pedro!",
    "A documentação poderia ser mais clara em alguns pontos.",
    "Instalei Python versão 3.11 conforme recomendado."
]

sentimento_chain = prompt_sentimento | llm

print("🔍 ANÁLISE DE SENTIMENTOS:")
print("=" * 50)

for texto in textos_teste:
    resultado = sentimento_chain.invoke({"texto": texto})
    sentimento = resultado.content.strip()
    
    # Emoji baseado no sentimento
    emoji = "😊" if "POSITIVO" in sentimento else "😞" if "NEGATIVO" in sentimento else "😐"
    
    print(f"{emoji} {sentimento}")
    print(f"   Texto: {texto[:60]}{'...' if len(texto) > 60 else ''}")
    print("-" * 50)

## 🧪 Exercício Prático: Seu Primeiro Template Complexo

Bora botar a mão na massa! Vou te desafiar a criar um template para um caso real.

**DESAFIO**: Criar um template para um **assistente de estudos** que:

1. Receba uma matéria e um tópico
2. Considere o nível de conhecimento do aluno
3. Gere um plano de estudos personalizado
4. Inclua recursos e dicas específicas

**Dica!** Pense em como você gostaria de ser ensinado e traduza isso para o template!

In [ ]:
# SEU CÓDIGO AQUI!
# Complete o template abaixo:

template_estudos = """
Você é um professor especialista em {materia}.

PERFIL DO ALUNO:
- Nível: {nivel}
- Tempo disponível: {tempo_disponivel}
- Estilo preferido: {estilo_aprendizado}

TÓPICO PARA ESTUDAR: {topico}

# COMPLETE AQUI SEU TEMPLATE!
# O que você gostaria que o assistente fizesse?
# Que estrutura seria útil?

"""

# Dica: pense em seções como:
# - Objetivos de aprendizado
# - Cronograma
# - Recursos recomendados
# - Exercícios práticos
# - Forma de avaliar o progresso

print("💪 Agora é sua vez! Complete o template acima.")

## 🔄 Conectando com os Próximos Módulos

Liiindo! Agora que você domina Prompt Templates, vamos ver como eles se conectam com o resto do curso:

### 📋 Módulo 5 - Output Parsers:
Os templates vão gerar textos que os **Output Parsers** vão estruturar em JSON, listas, objetos Python, etc.

### 🔗 Módulo 6 - Chains: 
Vamos combinar múltiplos templates em sequência para criar fluxos complexos.

### 💾 Módulo 7 - Memory Systems:
Os templates vão incluir variáveis que vêm da memória das conversas anteriores.

### 🤖 Módulo 11 - Agents:
Agents usam templates dinâmicos que mudam baseado nas ferramentas disponíveis!

**Dica!** Templates são a base de tudo no LangChain. Domine eles bem!

In [ ]:
# Preview do próximo módulo: Template + Output Parser
template_estruturado = """
Analise o seguinte produto e retorne APENAS um JSON com a estrutura:
{{
    "nome": "nome do produto",
    "categoria": "categoria principal",
    "preco_aproximado": "faixa de preço",
    "pontos_positivos": ["lista", "de", "pontos"],
    "pontos_negativos": ["lista", "de", "pontos"],
    "recomendacao": "sim/não/talvez"
}}

PRODUTO: {descricao_produto}
"""

prompt_preview = PromptTemplate(
    input_variables=["descricao_produto"],
    template=template_estruturado
)

# Testando
preview_chain = prompt_preview | llm
resultado = preview_chain.invoke({
    "descricao_produto": "Notebook gamer com RTX 4060, 16GB RAM, SSD 512GB, Intel i7, tela 144Hz por R$ 4.500"
})

print("🔮 PREVIEW Módulo 5 - Output estruturado:")
print(resultado.content)

## 📈 Medindo Performance dos Templates

Uma coisa importante: como saber se seu template está funcionando bem?

Vamos criar algumas métricas simples para avaliar nossos templates!

In [ ]:
import time
import json

def testar_template_performance(prompt_template, llm, casos_teste, nome_teste):
    """
    Testa a performance de um template com múltiplos casos
    """
    print(f"\n🔬 TESTANDO: {nome_teste}")
    print("=" * 50)
    
    chain = prompt_template | llm
    tempos = []
    sucessos = 0
    
    for i, caso in enumerate(casos_teste, 1):
        try:
            inicio = time.time()
            resultado = chain.invoke(caso)
            fim = time.time()
            
            tempo = fim - inicio
            tempos.append(tempo)
            sucessos += 1
            
            print(f"✅ Caso {i}: {tempo:.2f}s - {len(resultado.content)} caracteres")
            
        except Exception as e:
            print(f"❌ Caso {i}: Erro - {str(e)}")
    
    # Estatísticas
    if tempos:
        tempo_medio = sum(tempos) / len(tempos)
        print(f"\n📊 RESULTADOS:")
        print(f"   Sucessos: {sucessos}/{len(casos_teste)} ({sucessos/len(casos_teste)*100:.1f}%)")
        print(f"   Tempo médio: {tempo_medio:.2f}s")
        print(f"   Tempo total: {sum(tempos):.2f}s")
    
    return sucessos, tempos

print("⚡ Função de teste criada!")

In [ ]:
# Testando performance dos nossos templates
casos_teste_simples = [
    {"conceito": "machine learning", "idade": 12},
    {"conceito": "quantum computing", "idade": 25},
    {"conceito": "blockchain", "idade": 8}
]

casos_teste_sentimento = [
    {"texto": "Produto excelente, superou expectativas!"},
    {"texto": "Entrega foi feita conforme combinado."},
    {"texto": "Muito caro pelo que oferece, não recomendo."}
]

# Testando template simples
sucessos_simples, tempos_simples = testar_template_performance(
    prompt, llm, casos_teste_simples, "Template Simples"
)

# Testando template de sentimento
sucessos_sentimento, tempos_sentimento = testar_template_performance(
    prompt_sentimento, llm, casos_teste_sentimento, "Template Sentimento"
)

In [ ]:
# Visualizando comparação de performance
templates_nomes = ['Simples', 'Sentimento']
tempos_medios = [
    sum(tempos_simples) / len(tempos_simples) if tempos_simples else 0,
    sum(tempos_sentimento) / len(tempos_sentimento) if tempos_sentimento else 0
]
taxas_sucesso = [
    sucessos_simples / len(casos_teste_simples) * 100,
    sucessos_sentimento / len(casos_teste_sentimento) * 100
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico de tempos
bars1 = ax1.bar(templates_nomes, tempos_medios, color=['#FF6B6B', '#4ECDC4'])
ax1.set_title('⏱️ Tempo Médio de Resposta')
ax1.set_ylabel('Segundos')
ax1.grid(True, alpha=0.3)

# Gráfico de taxa de sucesso
bars2 = ax2.bar(templates_nomes, taxas_sucesso, color=['#45B7D1', '#96CEB4'])
ax2.set_title('✅ Taxa de Sucesso')
ax2.set_ylabel('Porcentagem (%)')
ax2.set_ylim(0, 100)
ax2.grid(True, alpha=0.3)

# Adicionando valores nas barras
for bar, valor in zip(bars1, tempos_medios):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{valor:.2f}s', ha='center', va='bottom')

for bar, valor in zip(bars2, taxas_sucesso):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{valor:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("📈 Análise concluída! Use essas métricas para otimizar seus templates.")

## 🎓 Exercício Final: Criando um Sistema de Templates

Agora é hora do desafio final! Vamos criar um **sistema de templates** que pode ser reutilizado em diferentes contextos.

**MISSÃO**: Criar templates para um chatbot educacional que:
1. Tenha diferentes "modos" (explicativo, quiz, resumo)
2. Adapte a linguagem ao nível do usuário
3. Mantenha consistência na personalidade

![](/imagens/langchain-modulo-04_img_04.png)

In [ ]:
# DESAFIO: Complete este sistema de templates educacionais

class SistemaTemplatesEducacional:
    def __init__(self):
        # Template base para manter consistência
        self.personalidade_base = "Você é o EduBot, um assistente educacional amigável e paciente."
        
        # COMPLETE: Crie templates para cada modo
        self.templates = {
            "explicativo": """
            {personalidade}
            
            MODO: Explicação Didática
            NÍVEL: {nivel}
            TÓPICO: {topico}
            
            # COMPLETE AQUI!
            """,
            
            "quiz": """
            {personalidade}
            
            MODO: Quiz Interativo
            # COMPLETE AQUI!
            """,
            
            "resumo": """
            {personalidade}
            
            MODO: Resumo Executivo
            # COMPLETE AQUI!
            """
        }
    
    def get_prompt(self, modo, **kwargs):
        # COMPLETE: Implemente a lógica para retornar o prompt correto
        pass

# Teste seu sistema aqui!
sistema = SistemaTemplatesEducacional()
print("🏗️ Sistema criado! Agora complete os templates.")

## 🎉 Resumo do Módulo: O que Aprendemos

**Liiindo!** Você acabou de dominar um dos pilares mais importantes do LangChain!

### ✅ Checklist do que você aprendeu:

1. **PromptTemplate básico** - Para casos simples e rápidos
2. **ChatPromptTemplate** - Para conversas estruturadas com roles
3. **Templates parciais** - Para reutilizar configurações padrão
4. **Few-shot learning** - Para ensinar padrões com exemplos
5. **Validação e performance** - Para garantir qualidade
6. **Integração com LCEL** - Conectando com chains usando `|`

### 🚀 Próximos passos:

- **Módulo 5**: Output Parsers - Estruturar as respostas
- **Módulo 6**: Chains - Combinar múltiplos templates
- **Módulo 7**: Memory - Templates que lembram do contexto

### 💡 Dica Final:

Templates são como receitas de bolo: quanto mais você pratica, melhores ficam! Comece simples, teste muito e vá evoluindo.

**Bora para o próximo módulo!** 🎯

In [ ]:
# Célula final: Preparando para o próximo módulo
print("🎊 PARABÉNS! Você completou o Módulo 4!")
print("\n📚 RECAP - Principais conceitos:")
print("   ✅ PromptTemplate - Templates básicos com variáveis")
print("   ✅ ChatPromptTemplate - Templates para conversas")
print("   ✅ Templates parciais - Reutilização inteligente")
print("   ✅ Few-shot learning - Aprendizado por exemplos")
print("   ✅ Performance e validação - Qualidade garantida")

print("\n🔜 PRÓXIMO MÓDULO: Output Parsers")
print("   🎯 Como estruturar as respostas da IA")
print("   🎯 JSON, listas, objetos Python automáticos")
print("   🎯 Validação e tratamento de erros")

print("\n💪 Continue praticando! Nos vemos no Módulo 5!")
print("\n" + "="*50)
print("Pedro Nunes Guth - LangChain v0.3 Course")
print("="*50)